In [4]:
using JuMP, Cbc

loc = 1:10

# x coordinate of each location
x_coord = [0 20 18 30 35 33 4 4 11 2]
# y coordinate of each location
y_coord = [0 20 10 12 0 25 27 10 0 16]

# calculate Euclidean distance between ever pair of locations
dist = Dict()
for i in loc
    for j in loc
        if i <= j
            dist[i,j] = sqrt((x_coord[i]-x_coord[j])^2 + (y_coord[i] - y_coord[j])^2)
        else
            # distance from i to j is the same as j to i
            dist[i,j] = dist[j,i]
        end
    end
end

# current cars at each location
supply = [6 11 4 8 12 2 14 11 15 7]

# required cars at each location
demand = [8 6 8 9 9 7 13 7 9 14]

## params for piecewise linear portion ##

# x interval end-points
a = [0 2 5 8]
# coef
m = [5 3 1]
# constant
b = [0 4 14]

mod = Model()

@variable(mod, x[loc,loc] >= 0, Int)
@variable(mod, t[loc,loc])
@variable(mod, y[loc, loc, 1:3], Bin)
@variable(mod, w[loc, loc, 1:3])

@objective(mod, Min, sum(t))

@constraint(mod, use_supply[i in loc], sum(x[i,j] for j in loc) == supply[i])
@constraint(mod, meet_demand[i in loc], sum(x[j,i] for j in loc) == demand[i])

@constraint(mod, t_equiv[i in loc, j in loc], sum(m[k]*dist[i,j]*w[i,j,k]+ b[k]*dist[i,j]*y[i,j,k] for k in 1:3) == t[i,j])

@constraint(mod, w_equiv[i in loc, j in loc], sum(w[i,j,k] for k in 1:3) == x[i,j])
@constraint(mod, choose_one[i in loc, j in loc], sum(y[i,j,k] for k in 1:3) == 1)

@constraint(mod, int_lb_init[i in loc, j in loc], w[i,j,1] >= a[1]*y[i,j,1])
@constraint(mod, int_lb[i in loc, j in loc, k in 2:3], w[i,j,k] >= a[k]*y[i,j,k])
@constraint(mod, int_ub[i in loc, j in loc, k in 1:3], w[i,j,k] <= a[k+1]*y[i,j,k])

set_optimizer(mod, Cbc.Optimizer)
set_silent(mod)

optimize!(mod)
println("Total cost: ", objective_value(mod))
println("strategy: ")
for i in loc
    for j in loc
        if i != j
            if value(x[i,j]) > 0.001
                println("Send ", value(x[i,j]), " cars from ", i, " to ", j)
            end
        end
    end
end

Total cost: 1619.8802337052107
strategy: 
Send 4.0 cars from 2 to 6
Send 1.0 cars from 2 to 7
Send 1.0 cars from 4 to 5
Send 1.0 cars from 4 to 6
Send 3.0 cars from 5 to 4
Send 1.0 cars from 5 to 9
Send 5.999999999999999 cars from 7 to 10
Send 5.0 cars from 8 to 10
Send 2.0 cars from 9 to 1
Send 4.0 cars from 9 to 3
Send 1.0 cars from 9 to 8
Send 4.0 cars from 10 to 7
